In [1]:
import spotipy as spot
import pandas as pd
import json

In [2]:
config = json.load(open(file="config.json", encoding="utf-8"))
config

{'CLIENT_ID': '468b8b024bfb41d5b1957dad2afc766a',
 'CLIENT_SECRET': '8827668f8ed64f13bf8c2e83781c3997',
 'REDIRECT_URI': 'http://localhost:8080'}

In [3]:
# Authorization Flow that WORKS
try:
    token = spot.util.prompt_for_user_token(username='frostcow',
        scope=["user-read-recently-played",
            "playlist-modify-public",
            "user-library-read",],
        client_id=config["CLIENT_ID"],
        client_secret=config["CLIENT_SECRET"],
        redirect_uri=config["REDIRECT_URI"])
    test_sp = spot.Spotify(auth=token)
except spot.SpotifyOauthError:
    print("User declined")

In [4]:
type(token)

str

# General purpose testing grounds for function alterations or new code below

In [5]:
from main import get_last_50_songs, get_saved_tracks

last = get_last_50_songs(test_sp)
saved = get_saved_tracks(test_sp, 2000)

In [6]:
# last.head()
saved.head()

,Track ID,Song Name,Album Name,Artist Name,Explicit,Song Popularity,Album Release Year
0,33Ze1sP9CuFFi1r6OxVzTP,Location on the Flyer,March Madness,ShooterGang Kony,True,54,2018.0
1,0ezwvY7TkCwDEkKR49084k,GANG GANG (with Lil Wayne),Hall of Fame,Polo G,True,63,2021.0
2,7lQyUSFLyCa8o4rBxsUNPU,top gun,bag or die,bbno$,True,57,2022.0
3,1VWCOx2EruZ7OqqiqWf7xe,Taking Over (ft. Miyoki),Get Mad,Rob Gasser,False,42,2014.0
4,2g7HN3Wz0j4YKV8udMcDMb,Prism,Prism,Case & Point,False,29,2014.0


In [7]:
saved["Album Release Year"].unique()

array([2018., 2021., 2022., 2014., 2016., 2019., 1997., 2013., 2006.,
       2017., 2020., 2010., 2011., 2009., 2012., 2004., 2015., 2008.,
       1990., 1991., 2001., 1970., 1977., 1999., 1971., 2007., 1993.,
       2003., 2005.,   nan, 1988., 2002., 1998., 1980., 1995., 1978.,
       1979., 1973., 2000., 1996., 1972.])

In [8]:
from content_based import ohe

o_saved, o_last = ohe(saved=saved, last=last)

In [9]:
o_last.head()

,Track ID,Song Name,Album Name,Artist Name,Song Popularity,Album Release Year,0
0,0ob3wzPe3R3NpJtUEz0wUQ,Sunnebovals,Sunnebovals,Christian Peterson-Bergling,60,2022,1.0
1,3tFmZIO4nrqHC5oPumnSnO,Cronopio,Cronopio,Franco Robert,57,2021,1.0
2,6dG6b98wdolLoHdFaAusxR,A New Path,A New Path,Damien Jeunet,66,2022,1.0
3,4GmJDwysMN5Et05ZOOBkQe,November Is Here,November is Here,Sylvio,58,2022,1.0
4,4iMQEdNXGMVlAbqsJGhvxx,"Waltz, Interrupted","Waltz, Interrupted",Piotr Wiese,57,2021,1.0


In [10]:
from content_based import encode_year

e_saved, e_last = encode_year(saved=saved, last=last)

In [11]:
e_last.head()

,Track ID,Song Name,Album Name,Artist Name,Explicit,Song Popularity,Year
0,0ob3wzPe3R3NpJtUEz0wUQ,Sunnebovals,Sunnebovals,Christian Peterson-Bergling,False,60,52
1,3tFmZIO4nrqHC5oPumnSnO,Cronopio,Cronopio,Franco Robert,False,57,51
2,6dG6b98wdolLoHdFaAusxR,A New Path,A New Path,Damien Jeunet,False,66,52
3,4GmJDwysMN5Et05ZOOBkQe,November Is Here,November is Here,Sylvio,False,58,52
4,4iMQEdNXGMVlAbqsJGhvxx,"Waltz, Interrupted","Waltz, Interrupted",Piotr Wiese,False,57,51


In [13]:
from main import add_audio_features

feat = add_audio_features(test_sp, saved, limit=2000)
feat.head()

,Track ID,Song Name,Album Name,Artist Name,Explicit,Song Popularity,Year,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,33Ze1sP9CuFFi1r6OxVzTP,Location on the Flyer,March Madness,ShooterGang Kony,True,54.0,48.0,0.705,0.421,6,-11.784,0,0.5070,0.607000,0.000000,0.108,0.528,99.889,175334,4
1,0ezwvY7TkCwDEkKR49084k,GANG GANG (with Lil Wayne),Hall of Fame,Polo G,True,63.0,51.0,0.752,0.563,3,-8.066,0,0.2090,0.146000,0.000000,0.103,0.551,145.050,178776,4
2,7lQyUSFLyCa8o4rBxsUNPU,top gun,bag or die,bbno$,True,57.0,52.0,0.572,0.881,2,-5.041,1,0.1420,0.027900,0.000031,0.342,0.784,160.182,141133,4
3,1VWCOx2EruZ7OqqiqWf7xe,Taking Over (ft. Miyoki),Get Mad,Rob Gasser,False,42.0,44.0,0.525,0.899,1,-2.305,0,0.1320,0.000784,0.205000,0.226,0.631,173.907,222325,4
4,2g7HN3Wz0j4YKV8udMcDMb,Prism,Prism,Case & Point,False,29.0,44.0,0.852,0.838,6,-5.323,1,0.0767,0.003500,0.838000,0.092,0.204,127.954,219699,4
